## Minimum Spanning Tree

Terminologies:
- ***Spanning Tree***: **Undirected graph** with positive edge **weights**, subgraph of G, **connected and acyclic**, **includes all the vertices** of G.
- **cut**: partition of the vertices of G into **two non-empty, disjointed** sets.
- **crossing edge**: edge that **connects** a vertex in on set with a vertex in the other.

Strategies(Greedy Algorithms):
1. Kruskal Algorithm(**O(E logE)**) WeightEdge-based, suitable for **sparse graph**
   - Sort the edges by ascending order of weights.
   - Add the smallest weight edge to the MST, unless it forms a cycle(use Disjoint Set to check)
   - Repeat until all vertices are included.
2. Prim Algorithm(**O(E logE)**): Vertex-based, suitable for **dense graph**
   - Start from an arbitrary vertex, mark added vertex as visited.
   - Use min-priority queue to updated the edges(contains only one visited vertex)
   - Repeat until all vertices are included.

In [99]:
from typing import List, TypeVar, Set, Dict, Tuple

T = TypeVar("T")

class WeightedEdge:
    def __init__(self, src:T, dest: T, weight: int):
        self.src: T = src
        self.dest:T = dest
        self.weight: int = weight

    def __str__(self) -> str:
        return f"{self.src} - {self.dest}: {self.weight}"
    
    def __repr__(self) -> str:
        return self.__str__()

class EdgeWeightedGraph:
    def __init__(self):
        self.graph: Dict[T, List[WeightedEdge]] = {}
        self.edges: List[WeightedEdge] = []

    def add_edge(self, src, dest, weight):
        edge = WeightedEdge(src, dest, weight)
        self.edges.append(edge)

        if src not in self.graph:
            self.graph[src] = []
        if dest not in self.graph:
            self.graph[dest] = []

        self.graph[src].append(edge)
        self.graph[dest].append(edge)


    def get_edges(self) -> List[WeightedEdge]:
        return self.edges
    
    def get_vertices(self) -> Set[T]:
        return self.graph.keys()
    
    def get_adjacent_edges(self, v:T) -> List[WeightedEdge]:
        return self.graph[v]
    
    def get_graph(self) -> Dict[T, List[WeightedEdge]]:
        return self.graph
    


In [100]:
class DisjointSet:
    def __init__(self, vertices: List[T]):
        self.parent:Dict[T, T] = {v : v for v in vertices}
        self.rank: Dict[T, int] = {v: 1 for v in vertices}

    def find(self, v:T) -> T:
        if self.parent[v] != v:
            self.parent[v] = self.find(self.parent[v]) # Path Compression
        return self.parent[v]
    
    def union(self, v:T, w:T):
        v_parent = self.find(v)
        w_parent = self.find(w)

        if v_parent == w_parent:
            return
        
        if self.rank[v_parent] < self.rank[w_parent]:
            self.parent[v_parent] = w_parent
            self.rank[w_parent] += self.rank[v_parent]
        else: 
            self.parent[w_parent] = v_parent
            self.rank[v_parent] += self.rank[w_parent]

### Kruskal Algorithm

1. Sort the edges in ascending order
2. while (mst.size < V-1):  
   3. append edge, unless it forms a cycle 


In [101]:
class Kruskal:
    def __init__(self, ewg:EdgeWeightedGraph):
        self.edges:List[WeightedEdge] = ewg.get_edges()
        self.ds: DisjointSet = DisjointSet(ewg.get_vertices())
        self.vertices_num : int = len(ewg.get_vertices())

    def asc_sort_edges(self):
        self.edges.sort(key= lambda edge: edge.weight)

    def findMST(self) -> List[WeightedEdge]:
        mst:List[WeightedEdge] = []

        # Step 1: Sort the edges in ascending order
        self.asc_sort_edges()

        for edge in self.edges:
            src, dest = edge.src, edge.dest

            # Step 2: Check if the cycle is formed via Disjoint Set
            if self.ds.find(src) != self.ds.find(dest):
                mst.append(edge)
                self.ds.union(src, dest)
            
            # Step 3: If the number of edges in MST is equal to V-1, then break
            if len(mst) >= self.vertices_num - 1:
                break

        return mst


### Prim Algorithm 


In [102]:
import heapq

class MinPQ:
    def __init__(self):
        self.pq: List[Tuple[int, T, WeightedEdge]] = []

    def push(self, weight: int, vertex: T, edge: WeightedEdge):
        heapq.heappush(self.pq, (weight, vertex, edge))

    def pop(self) -> Tuple[int, T, WeightedEdge]:
        return heapq.heappop(self.pq)
    
    def is_empty(self) -> bool:
        return len(self.pq) == 0 

class Prim:
    def __init__(self, ewg:EdgeWeightedGraph): 
        self.graph = ewg
        self.start_vertex = list(ewg.get_vertices())[0]

    def findMST(self) -> List[WeightedEdge]:
        mst: List[WeightedEdge] = []
        visited: Set[T] = set()
        min_pq: MinPQ = MinPQ()

        min_pq.push(0, self.start_vertex, None)

        while not min_pq.is_empty() or len(visited) < len(self.graph.get_vertices()):
            weight, vertex, edge = min_pq.pop()

            if vertex in visited:
                continue
            
            visited.add(vertex)
            if edge:
                mst.append(edge)

            for adj_edge in self.graph.get_adjacent_edges(vertex):
                v = adj_edge.src if adj_edge.src != vertex else adj_edge.dest
                w = adj_edge.weight
                if v not in visited:
                    min_pq.push(w, v, adj_edge)

        return mst
        


In [103]:
def test():
    ewg = EdgeWeightedGraph()

    ewg.add_edge("A", "C",  26) #
    ewg.add_edge("A", "G",  58)
    ewg.add_edge("E", "A",  38)
    ewg.add_edge("G", "C",  40) #
    ewg.add_edge("H", "A",  16) #
    ewg.add_edge("H", "C",  36)
    ewg.add_edge("D", "C",  17) #
    ewg.add_edge("D", "B",  29)
    ewg.add_edge("B", "C",  34) 
    ewg.add_edge("B", "H",  19) #
    ewg.add_edge("B", "F",  42)
    ewg.add_edge("H", "F",  28) #
    ewg.add_edge("E", "F",  35) #
    ewg.add_edge("E", "H",  37)
    ewg.add_edge("E", "G",  93) 


    kruskal = Kruskal(ewg)
    prim = Prim(ewg)

    
    res_k = kruskal.findMST()
    res_p = prim.findMST()
    
    print(res_k)
    print(res_p) 


test()


[H - A: 16, D - C: 17, B - H: 19, A - C: 26, H - F: 28, E - F: 35, G - C: 40]
[H - A: 16, B - H: 19, A - C: 26, D - C: 17, H - F: 28, E - F: 35, G - C: 40]
